In [6]:
######################################################################
from sage.schemes.elliptic_curves.hom_scalar import EllipticCurveHom_scalar
from sage.schemes.elliptic_curves.isogeny_small_degree import isogenies_prime_degree
from sage.rings.number_field.number_field_morphisms import EmbeddedNumberFieldMorphism
######################################################################
#
#
#
#
#
#
#
#
#
#
#
########################################################################
#  ____         ____  _                                        
# |  _ \  ___  |  _ \| |__   __ _ _ __ ___                     
# | | | |/ _ \ | |_) | '_ \ / _` | '_ ` _ \                    
# | |_| |  __/ |  _ <| | | | (_| | | | | | |                   
# |____/ \___| |_| \_\_| |_|\__,_|_|_|_| |_| _   _             
#  / ___|___  _ __ ___  _ __  _   _| |_ __ _| |_(_) ___  _ __  
# | |   / _ \| '_ ` _ \| '_ \| | | | __/ _` | __| |/ _ \| '_ \ 
# | |__| (_) | | | | | | |_) | |_| | || (_| | |_| | (_) | | | |
#  \____\___/|_| |_| |_| .__/ \__,_|\__\__,_|\__|_|\___/|_| |_|
#                      |_|                                     
########################################################################
def makeDiscriminantFundamental(D):
    x = polygen(ZZ,'x')
    K.<a>=NumberField(x^2-D)
    return K.discriminant()
def findGenerator(D): #finds generator pi of order with discriminant D.
    DD = makeDiscriminantFundamental(D)
    x = polygen(ZZ, 'x')
    K.<a> = NumberField(x^2-DD)
    f = int(sqrt(D/DD))
    alpha = K.order_of_conductor(f).gens()[1]
    return alpha, K
def findIsogenyPrime(E): #returns a prime which is the degree of some CM-isogeny of E, whose eigenvalue of dx/y is not real.
    if not E.has_cm():
        return None
    else:
        D = E.cm_discriminant()
        alpha, K = findGenerator(D)
        a = 0
        b = 0
        while True:
            for b in range(0,a+1):
                p = (a+b*alpha).norm()
                if p>2 and p.is_prime():
                    return (p, a+b*alpha, K)
            a += 1
def CMExtend(E,F,p): #Extends E/F to a base field L over which E has an isogeny of degree p.
    R.<x> = PolynomialRing(F)
    f = x^2 - E.cm_discriminant()
    L.<b> = F.extension(f)
    E = E.base_extend(L)
    return (E,L)
def CMIsogeny(E,F): #Returns a CM Isogeny/L, phi:L->L (with CM, I mean that it is in End(E/L)\Z)
    p, omega, K = findIsogenyPrime(E)
    E, L = CMExtend(E,F,p)
    psis = isogenies_prime_degree(E,p)
    phi = psis[0]
    for psi in psis:
        if psi.codomain().j_invariant() == E.j_invariant():
            phi = psi
            break
    iota = phi.codomain().isomorphism_to(E)
    phi = iota*phi
    return (omega, K, phi,L)
def weighingFactorNonHolomorphicDifferential(E,a2,F): #CM Info on xdx/y
    omega, K, phi, L = CMIsogeny(E,F)
    deg_ = phi.degree()
    alpha_ = phi.scaling_factor()
    return (omega, K, deg_/alpha_,L)
def weighingFactorHolomorphicDifferential(E,a2,F): #CM Info on dx/y
    omega, K, phi, L = CMIsogeny(E,F)
    poly = list(phi.kernel_polynomial())
    n = len(poly) - 1
    alpha_ = phi.scaling_factor()
    deg_ = phi.degree()
    sum_ = (-1)*2*poly[-2]+a2/3*(deg_-1)
    return (omega,K,deg_/alpha_*a2/3-1/alpha_*sum_-a2/3*alpha_,L,alpha_)
def actionOnNonHolomorphic(E,a2,F):
    omega, K, c1, L = weighingFactorNonHolomorphicDifferential(E,a2,F)
    c2, alpha_ = weighingFactorHolomorphicDifferential(E,a2,F)[2],weighingFactorHolomorphicDifferential(E,a2,F)[-1]
    str_ = "Statement of CM: The action of CM"
    str_ += " on xdx/y"
    print(str_)
    str_ = ""
    str_ += "is given by (" + str(c1) + ") xdx/y + (" + str(c2) + ") dx/y"
    str_ += " where b is given by "
    print(str_)
    print(L)
    str_ = ""
    str_ += "Furthermore, the action above on dx/y is given by "
    str_ += str(alpha_)
    print(str_)
def diagonalizeDeRham(E,a2,F):
    omega, K, c1, L = weighingFactorNonHolomorphicDifferential(E,a2,F)
    c2, alpha_ = weighingFactorHolomorphicDifferential(E,a2,F)[2],weighingFactorHolomorphicDifferential(E,a2,F)[-1]
    str_ = "The diagonalized deRham Cohomology is given by the basis elements "
    A = Matrix([[alpha_,c2],[0,c1]])
    A_ = A.eigenvectors_right()
    str_ += "(" + str(A_[0][1][0][0]) + ")*dx/y + (" + str(A_[0][1][0][1]) + ")*xdx/y and "
    str_ += "(" + str(A_[1][1][0][0]) + ")*dx/y + (" + str(A_[1][1][0][1]) + ")*xdx/y"
    print(str_)
########################################################################
#  ____         ____  _                                        
# |  _ \  ___  |  _ \| |__   __ _ _ __ ___                     
# | | | |/ _ \ | |_) | '_ \ / _` | '_ ` _ \                    
# | |_| |  __/ |  _ <| | | | (_| | | | | | |                   
# |____/ \___| |_| \_\_| |_|\__,_|_|_|_| |_| _   _             
#  / ___|___  _ __ ___  _ __  _   _| |_ __ _| |_(_) ___  _ __  
# | |   / _ \| '_ ` _ \| '_ \| | | | __/ _` | __| |/ _ \| '_ \ 
# | |__| (_) | | | | | | |_) | |_| | || (_| | |_| | (_) | | | |
#  \____\___/|_| |_| |_| .__/ \__,_|\__\__,_|\__|_|\___/|_| |_|
#                      |_|                                     
########################################################################

In [8]:
#In order to use the above code, define your number field (or use QQ if that is where E is defined)
#Define your elliptic curve E
#In the function diagonalizeDeRham, the arguments are Elliptic Curve, a2 of elliptic curve, and the third arugment is the number field.
F.<c> = NumberField(x^2-2)
l0 = 1/2-3*c/8
E = EllipticCurve([0,0,0,-(1+14*l0+l0^2)/48,(1-33*l0-33*l0^2+l0^3)/(216*4)])
print(E.has_cm())
diagonalizeDeRham(E,0,F)

True
The diagonalized deRham Cohomology is given by the basis elements (1)*dx/y + (0)*xdx/y and (1)*dx/y + (-48*c - 64)*xdx/y
